# Выбор локации для скважины

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. По
Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st
import warnings
warnings.filterwarnings('ignore')

RS = 12345

In [2]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

Для удобства в дальнейшей работе создадим список, включающий в себя исходные данные:

In [3]:
dfs = [df_0, df_1, df_2]

Создадим функцию `data_info`, выводящую на экран информацию о таблице:

In [4]:
def data_info(data):
    print(f'Выведем первые 5 строк таблицы:')
    display(data.head())
    print(f'Выведем последние 5 строк таблицы:')
    display(data.tail())
    print(f'Выведем случайные 5 строк таблицы:')
    display(data.sample(5, random_state=RS))
    print(f'Выведем основную информацию о таблице:')
    data.info()
    print()
    print(f'Выведем описательные статистики таблицы:')
    display(data.describe().T)
    print(f'Проверим таблицу на пропуски: ')
    display(data.isna().sum())
    print(f'Проверим таблицу на дубликаты: ')
    display(data.duplicated().sum())

Рассмотрим основную информацию о таблицах:

In [5]:
for i in range(len(dfs)):
    print(f'Информация о таблице с данными для {i + 1}-го региона: \n ')
    data_info(dfs[i])

Информация о таблице с данными для 1-го региона: 
 
Выведем первые 5 строк таблицы:


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Выведем последние 5 строк таблицы:


,id,f0,f1,f2,product
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764
99999,1CWhH,1.764754,-0.266417,5.722849,149.633246


Выведем случайные 5 строк таблицы:


...

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.002023,1.732045,-8.760004,-1.162288,0.009424,1.158535,7.238262
f1,100000.0,-0.002081,1.730417,-7.084020,-1.174820,-0.009482,1.163678,7.844801
f2,100000.0,2.495128,3.473445,-11.970335,0.130359,2.484236,4.858794,16.739402
product,100000.0,95.000000,44.749921,0.000000,59.450441,94.925613,130.595027,190.029838


Проверим таблицу на пропуски: 


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Проверим таблицу на дубликаты: 


0

Согласно документации к данным:
* **id** — уникальный идентификатор скважины;
* **f0, f1, f2** — три признака точек (неважно, что они означают, но сами признаки значимы);
* **product** — объём запасов в скважине (тыс. баррелей).

Столбец `id` не несет какой-либо полезной информации - удалим его:

In [6]:
for df in dfs:
    df.drop(['id'], axis=1, inplace=True)

### Вывод

Предварительно можно утверждать, что имеющихся данных **достаточно** для построения модели машинного обучения и дальнейшего исследования.

Данные для каждого региона полные, дубликаты отсутствуют.

## Обучение и проверка модели

Создадим функцию `data_split`, возвращающую словарь с *train* и *valid* данными для их дальнейшего применения:

In [7]:
def data_split(data):
    data_train, data_valid = train_test_split(data, test_size = 0.25, random_state=RS)
    
    features_train = data_train.drop(['product'], axis=1)
    target_train = data_train['product']
    
    features_valid = data_valid.drop(['product'], axis=1)
    target_valid = data_valid['product']
    
    
    return {'features_train': features_train,
            'target_train': target_train,
            'features_valid': features_valid,
            'target_valid': target_valid,
            'data_train': data_train,
            'data_valid': data_valid
           }

Создадим функцию `model_predict`, которая обучает модель и возвращает массив с предсказаниями:

In [8]:
def model_predict(features_train, target_train, features_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    return predictions

Создадим функцию `model_analysis`, выводящую на экран значение метрики **RMSE** для обученной модели и средний запас предсказанного сырья:

In [9]:
def model_analysis(data):
    features_valid = data_split(data).get('features_valid')
    target_valid = data_split(data).get('target_valid')
    
    features_train = data_split(data).get('features_train')
    target_train = data_split(data).get('target_train')
    
    predictions = model_predict(features_train, target_train, features_valid)
    
    print(f'Средний запас предсказанного сырья = {predictions.mean()}')
    print(f'RMSE = {mean_squared_error(target_valid, predictions) ** 0.5}')
    print()

Исследуем каждый регион:

In [10]:
for i in range(len(dfs)):
    print(f'Информация для {i + 1}-го региона: ')
    model_analysis(dfs[i])

Информация для 1-го региона: 
Средний запас предсказанного сырья = 92.59256778438038
RMSE = 37.5794217150813

Информация для 2-го региона: 
Средний запас предсказанного сырья = 68.728546895446
RMSE = 0.893099286775616

Информация для 3-го региона: 
Средний запас предсказанного сырья = 94.96504596800489
RMSE = 40.02970873393434



Заполним список `predictions` массивами, содержащими предсказания модели каждого региона:

In [11]:
predictions = [model_predict(data_split(df).get('features_train'),
                  data_split(df).get('target_train'),
                  data_split(df).get('features_valid')
                  ) for df in dfs]

### Вывод

Исследование показывает, что наибольшее среднее значение запаса предсказанного сырья у **3-го** региона - **95 тыс. бар.**. Однако данный регион имеет и наибольший разброс (метрика **RMSE**) равный **40 тыс. бар.**.

В свою же очередь **2-й** регион показывает наименьшее значение среднего запаса - всего лишь **69 тыс. бар.**. Но разброс значительно меньше относительно других регионов, что позволяет более объективно судить о регионе. Он составляет приблизительно **1 тыс. бар.**

## Подготовка к расчёту прибыли

Согласно условиям задачи:
* При разведке региона исследуют **500** точек, из которых с помощью машинного обучения выбирают **200 лучших** для разработки.
* Бюджет на разработку скважин в регионе — **10 млрд рублей**.
* При нынешних ценах один баррель сырья приносит **450 рублей** дохода. Доход с каждой единицы продукта составляет **450 тыс. рублей**, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше **2.5%**. Среди них выбирают регион с наибольшей средней прибылью.

Запишем основные константы для расчёта прибыли:

In [12]:
BUDGET = 10 ** 10
RUB_PER_BAR = 450
RUB_PER_IU = RUB_PER_BAR * 1000
POINTS_FOR_BUDGET = 200
PROB_RISK = 0.025

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины:

In [13]:
print(f'{round(BUDGET / (RUB_PER_IU * POINTS_FOR_BUDGET))} тыс. бар.')

111 тыс. бар.


### Вывод

Расчёт показал, что достаточный объем сырья для безубыточной разработки новой скважины составляет порядка **111 тыс. бар.**. Однако ни один из регионов в среднем не показывает такого объема.

## Расчёт прибыли и рисков 

Создадим функцию `revenue`, возвращающую значение прибыли:

In [14]:
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return (RUB_PER_IU * selected.sum() - BUDGET) / 10 ** 9

Заполним список `df_valid` *valid* данными для дальнейшего использования:

In [15]:
df_valid = [data_split(df).get('data_valid') for df in dfs]

Рассчитаем прибыль для каждого региона:

In [16]:
for i in range(len(dfs)):
    df_valid[i]['predictions'] = predictions[i]
    print(f'Прибыль для {i + 1}-го региона:')
    print((revenue(data_split(dfs[i]).get('target_valid'), df_valid[i]['predictions'], 200)), 'млрд. руб.', '\n')

Прибыль для 1-го региона:
3.3208260431398524 млрд. руб. 

Прибыль для 2-го региона:
2.4150866966815108 млрд. руб. 

Прибыль для 3-го региона:
2.7103499635998327 млрд. руб. 



In [17]:
state = np.random.RandomState(12345)

Создадим функцию `bootstrap`, применяющую данную технику к данным:

In [18]:
def bootstrap(target, predictions):
    values = []
    
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample, POINTS_FOR_BUDGET))
    
    values = pd.Series(values)
    mean = values.mean()
    confidence_interval = [values.quantile(PROB_RISK).round(2), values.quantile(1 - PROB_RISK).round(2)]
    print(f'Среднее значение прибыли: {mean}, млрд. руб.')
    print(f'Доверительный интервал: {confidence_interval}')
    print(f'Риски убытков: {(values < 0).mean():.0%}')
    print()

Получим информацию для каждого региона:

In [19]:
for i in range(len(dfs)):
    print(f'Информация для 200 случайных скважин из {i + 1}-го региона: ')
    bootstrap(data_split(dfs[i]).get('target_valid'), df_valid[i]['predictions'])

Информация для 200 случайных скважин из 1-го региона: 
Среднее значение прибыли: 0.42593852691059236, млрд. руб.
Доверительный интервал: [-0.1, 0.95]
Риски убытков: 6%

Информация для 200 случайных скважин из 2-го региона: 
Среднее значение прибыли: 0.5182594936973248, млрд. руб.
Доверительный интервал: [0.13, 0.95]
Риски убытков: 0%

Информация для 200 случайных скважин из 3-го региона: 
Среднее значение прибыли: 0.4201940053440501, млрд. руб.
Доверительный интервал: [-0.12, 0.99]
Риски убытков: 6%



### Вывод

Проведя анализ полученных данных, можно сделать вывод о том, что наиболее переспективным будет использование **2-го** региона.

На примере 200 точек с лучшими показателями прибыли, выбранных из 500 случайных точек были получены наибольшие значения прибыли - **520 млн. руб** и наименьший риск убытков, стремящийся к **нулю**.

## Общий вывод

Для данного исследования была использована модель **Логистической регрессии**.

Значение достаточного объёма сырья для безубыточной разработки новой скважины: **111 тыс. бар.**

Значения прибыли для каждого региона:
* 1-й регион: **3.3 млрд. руб.**
* 2-й регион: **2.4 млрд. руб.**
* 3-й регион: **2.7 млрд. руб.**

Для расчета прибыли и рисков в 200 лучших из 500 случайных точек была использована техника **Bootstrap**.

Наилучший результат показал **второй** регион, имеющий наибольшее значение средней прибыли, порядка **500 млн. руб** и наименьшее значение риска убытков, стремящегося к **нулю**. Остальные показатели регионов:
* 1-й регион: Среднее значение прибыли - **418 млн. руб.**, Риски убытков - **7%**
* 3-й регион: Среднее значение прибыли - **428 млн. руб.**, Риски убытков - **6%**